# Chains in LangChain
* LLMChain
* Sequential Chains
  --- combine multiple chains where the output of the one chain is the input of the next chain
  * SimpleSequentialChain (single i/o)
  * SequentialChain (multiple i/o)
* Router Chain

![Simple Sequential Chain](./images/ssc.png)
![Sequential Chain](./images/sc.png)
![Router Chain](./images/rc.png)

In [1]:
import warnings
warnings.filterwarnings('ignore')

model = 'gemma:2b'

In [4]:
# %pip install pandas

In [45]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head(2)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."


## LLMChain

In [9]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [10]:
llm = Ollama(model=model,temperature=0.9)

In [12]:
prompt = PromptTemplate.from_template("What is the best name to describe a company that makes {product}?")

In [13]:
chain = LLMChain(llm=llm, prompt=prompt)

In [16]:
product = "Queen Size Sheet Set"
print(chain.run(product))

The best name to describe a company that makes Queen Size Sheet Set would depend on the company's focus, target audience, and overall brand image. Here are some ideas to consider:

**Descriptive Names:**

* **The Pillow Palace**
* **Queen Serenity Sleepwear**
* **Pillow Perfect**
* **The Serene Sleep Shop**
* **The Cozy Castle**

**Catchy Names:**

* **Sweet Dreams Sleep Systems**
* **The Sleep Sanctuary**
* **The Pillow Place**
* **The Bedtime Bazaar**
* **Sleepwell Solutions**

**Brand-Specific Names:**

* **The Queen's Choice**
* **Simply Queen**
* **The Sleep Goddess**
* **The Serta Sleep Suite**
* **The Sweet Sleep Queen**

**Unique Names:**

* **The Dream Weaver**
* **Circadian Sleep Solutions**
* **The Midnight Muse**
* **Dreamscapes Bedding**
* **The Sleep Symphony**

**Additional factors to consider:**

* **Target audience:** Is the focus on high-end luxury, budget-conscious shoppers, or a specific health-conscious group?
* **Brand image:** Is the company playful and fun, or s

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

In [18]:
# prompt template 1
first_prompt = PromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:
# prompt template 2
second_prompt = PromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one,chain_two],verbose=True)

In [21]:
print(overall_simple_chain.run(product))



> Entering new SimpleSequentialChain chain...
* The Sleep Foundation
* The Ultimate Bedding Company
* SleepTime Bedding
* SleepTime Sheet Sets
* Bedsheet Bliss
* The Pillow Nook
**The Sleep Foundation**

The Sleep Foundation is a leading provider of high-quality bedding products for all ages. We offer a wide range of mattresses, pillows, sheets, and other sleep accessories to help you get the restful sleep you deserve.

> Finished chain.
**The Sleep Foundation**

The Sleep Foundation is a leading provider of high-quality bedding products for all ages. We offer a wide range of mattresses, pillows, sheets, and other sleep accessories to help you get the restful sleep you deserve.


## SequentialChain

In [22]:
from langchain.chains import SequentialChain

In [23]:
# prompt template 1: translate to english
first_prompt = PromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,output_key="English_Review")

In [24]:
second_prompt = PromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,output_key="summary")


In [25]:
# prompt template 3: translate to english
third_prompt = PromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,output_key="language")


In [26]:
# prompt template 4: follow up message
fourth_prompt = PromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,output_key="followup_message")

In [32]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message","language"],
    verbose=True
)

In [33]:
review = df.Review[5]
sequentialOutput = overall_chain(review)
print(sequentialOutput)



> Entering new SequentialChain chain...

> Finished chain.
{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?", 'English_Review': 'The review is complaining about the quality of the product. The mousse does not live up to expectations and is considered bizarre. The customer bought the same product at the store and at home and the taste is better at the store. They believe it is a counterfeit or a fake.', 'summary': 'Sure, here is a summary:\n\nThe review describes the product as bizarre and lower quality than expected, suggesting it might be counterfeit or fake.', 'followup_message': "Bien sûr, voici un suivi :\n\nLe résumé décrit le produit comme bizarre et de qualité inférieure à ce que l'on pouvait attendre, suggérant qu'il s'agit d'un faux ou d'un imité.\n\nJe suis en train de examiner attentivement le produit et je suis prêt à partager mon interprétation de

## Router Chain

In [34]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [35]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

# description is used for LLMROUTERChain

In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [37]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [38]:
default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [39]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [40]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), # which sub_chain to route between
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [41]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True)

In [42]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the difference between light and matter?'}
> Finished chain.


'Light and matter are two of the most fundamental concepts in physics. While they are often used interchangeably, there are some key differences between the two.\n\nLight is a form of energy that can travel through space at a constant speed, regardless of the medium it is traveling through. Matter, on the other hand, is anything that has mass and can therefore exert force on other objects.\n\nAnother key difference between light and matter is the fact that light can travel through some materials, such as water and glass, while matter is much more resistant to this. This is because light waves have a different wavelength than matter waves, which allows them to penetrate deeper into the material.\n\nDespite their differences, light and matter are closely related. The speed of light in a vacuum is the same as the speed of light in any medium, regardless of the properties of the medium. This means that light can travel through matter at the same speed as it travels through a vacuum.\n\nIn 

In [49]:
chain.run("what is 2 plus 2")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is 2 plus 2?'}
> Finished chain.


'2 + 2 = 4.\n\nI do not know the answer to this question since it is a trick question that plays on your expectations.'

In [46]:
chain.run("Why does every cell in our human body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Explain the structure of DNA in detail and how it relates to the function of cells.'}
> Finished chain.


"Sure, here's a detailed explanation of the structure of DNA and its relation to the function of cells:\n\n**The Structure of DNA**\n\nDNA is a double-stranded molecule composed of two polynucleotide strands. It is found in the nucleus of cells and is responsible for storing and transmitting genetic information.\n\nThe DNA molecule is made up of two complementary strands of nucleotides. Each strand is composed of a sugar molecule (deoxyribose) backbone and a nitrogenous base pair. The bases are arranged in a specific order along the backbone. There are four different bases: adenine (A), cytosine (C), guanine (G), and thymine (T).\n\nThe DNA molecule can be arranged in different ways to form different structures. Two common structures are A-DNA and Z-DNA.\n\n* **A-DNA** is a right-handed double helix with a regular arrangement of bases. The bases are arranged in a regular pattern, with A always next to T and C always next to G.\n* **Z-DNA** is a left-handed double helix with a irregular